In [107]:
import pandas as pd
import requests
from io import StringIO
import re

## Q1 Withdrawn IPOs by Company Type

In [108]:
url = "https://stockanalysis.com/ipos/withdrawn/"

# Use requests to fetch the page
headers = {"User-Agent": "Mozilla/5.0"}  # helps bypass bot-blockers
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    tables = pd.read_html(response.text)
    df_ipo_withdrawn = tables[0]  # Adjust index if needed
    print(df_ipo_withdrawn.head())
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")

  Symbol                Company Name    Price Range Shares Offered
0   ODTX  Odyssey Therapeutics, Inc.              -              -
1   UNFL      Unifoil Holdings, Inc.  $3.00 - $4.00        2000000
2   AURN        Aurion Biotech, Inc.              -              -
3   ROTR             PHI Group, Inc.              -              -
4    ONE           One Power Company              -              -


In [109]:
df_ipo_withdrawn.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.2+ KB


In [110]:
# Assume df_ipo_withdrawn already exists and has a 'Company' column
def classify_company(name):
    name = name.lower()
    if "acquisition corp" in name or "acquisition corporation" in name:
        return "Acq.Corp"
    elif "inc" in name or "incorporated" in name:
        return "Inc"
    elif "group" in name:
        return "Group"
    elif "ltd" in name or "limited" in name:
        return "Limited"
    elif "holdings" in name:
        return "Holdings"
    else:
        return "Other"
    
    # Apply the classification
df_ipo_withdrawn["Company Class"] = df_ipo_withdrawn["Company Name"].apply(classify_company)

In [111]:
df_ipo_withdrawn.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other


In [112]:
# calculate the average price from the 'Price Range' column
def parse_avg_price(price_range):
    if pd.isna(price_range):
        return None
    # Extract decimal numbers from the string
    numbers = re.findall(r"\d+\.\d+", price_range)
    if len(numbers) == 2:
        avg = (float(numbers[0]) + float(numbers[1])) / 2
        return round(avg, 1)
    elif len(numbers) == 1:
        return round(float(numbers[0]), 1)
    else:
        return None

# Apply to the DataFrame
df_ipo_withdrawn["Avg. price"] = df_ipo_withdrawn["Price Range"].apply(parse_avg_price)

# Preview the result
print(df_ipo_withdrawn[["Price Range", "Avg. price"]].head())


     Price Range  Avg. price
0              -         NaN
1  $3.00 - $4.00         3.5
2              -         NaN
3              -         NaN
4              -         NaN


In [113]:
# Clean and convert 'Shares Offered' to numeric
df_ipo_withdrawn["Shares Offered"] = (
    df_ipo_withdrawn["Shares Offered"]
    .astype(str)  # Ensure string type to apply string operations
    .str.replace(",", "", regex=False)  # Remove commas
)

# Convert to numeric, coercing errors to NaN
df_ipo_withdrawn["Shares Offered"] = pd.to_numeric(df_ipo_withdrawn["Shares Offered"], errors="coerce")

# Preview the cleaned column
df_ipo_withdrawn[["Shares Offered"]].head()

,Shares Offered
0,NaN
1,2000000.0
2,NaN
3,NaN
4,NaN


In [114]:
df_ipo_withdrawn['Withdrawn Value'] = df_ipo_withdrawn['Shares Offered'] * df_ipo_withdrawn['Avg. price']
df_ipo_withdrawn.head()

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price,Withdrawn Value
0,ODTX,"Odyssey Therapeutics, Inc.",-,NaN,Inc,NaN,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000.0,Inc,3.5,7000000.0
2,AURN,"Aurion Biotech, Inc.",-,NaN,Inc,NaN,NaN
3,ROTR,"PHI Group, Inc.",-,NaN,Inc,NaN,NaN
4,ONE,One Power Company,-,NaN,Other,NaN,NaN


In [115]:
df_ipo_withdrawn['Withdrawn Value'].notna().sum()

71

In [116]:
df_ipo_withdrawn.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Symbol           100 non-null    object 
 1   Company Name     100 non-null    object 
 2   Price Range      100 non-null    object 
 3   Shares Offered   72 non-null     float64
 4   Company Class    100 non-null    object 
 5   Avg. price       73 non-null     float64
 6   Withdrawn Value  71 non-null     float64
dtypes: float64(3), object(4)
memory usage: 5.6+ KB


In [117]:
df_ipo_withdrawn.groupby('Company Class')['Withdrawn Value'].sum().apply(lambda x: f'{x:,.0f}')

Company Class
Acq.Corp    4,021,000,000
Group          33,600,000
Holdings       75,000,000
Inc         2,257,023,566
Limited       549,856,002
Other         767,697,777
Name: Withdrawn Value, dtype: object

## Question 2: Median Sharpe Ratio for 2024 IPOs (First 5 Months)

In [118]:
import pandas as pd
import requests
from io import StringIO
import re

In [119]:
url = "https://stockanalysis.com/ipos/2024/"

# Use requests to fetch the page
headers = {"User-Agent": "Mozilla/5.0"}  # helps bypass bot-blockers
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    tables = pd.read_html(response.text)
    df_ipo_2024 = tables[0]  # Adjust index if needed
    print(df_ipo_2024.head())
else:
    print(f"Failed to retrieve page. Status code: {response.status_code}")

       IPO Date Symbol                                 Company Name IPO Price  \
0  Dec 31, 2024   ONEG                OneConstruction Group Limited     $4.00   
1  Dec 27, 2024    PHH      Park Ha Biological Technology Co., Ltd.     $4.00   
2  Dec 23, 2024    HIT                         Health In Tech, Inc.     $4.00   
3  Dec 23, 2024   TDAC  Translational Development Acquisition Corp.    $10.00   
4  Dec 20, 2024   RANG              Range Capital Acquisition Corp.    $10.00   

  Current   Return  
0   $5.02   25.00%  
1  $29.39  564.25%  
2   $0.64  -84.40%  
3  $10.27    2.70%  
4  $10.20    2.00%  


In [120]:
df_ipo_2024['IPO Date'] = pd.to_datetime(df_ipo_2024['IPO Date'], errors='coerce')
df_ipo_filtered = df_ipo_2024[df_ipo_2024['IPO Date'] <= '2024-06-01'].reset_index(drop=True)


In [121]:
ALL_TICKERS = df_ipo_filtered['Symbol'].unique()
len(ALL_TICKERS)

77

In [122]:
import time
import yfinance as yf
import numpy as np

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)

  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 NAKA
1 BOW
2 HDL
3 RFAI
4 JDZG
5 RAY
6 BTOC
7 ZK
8 GPAT
9 PAL
10 SVCO
11 NNE
12 CCIX
13 VIK
14 ZONE
15 LOAR
16 MRX
17 RBRK
18 NCI
19 MFI
20 YYGH
21 TRSG
22 CDTG
23 CTRI
24 IBTA
25 MTEN
26 SUPX
27 TWG
28 ULS
29 PACS
30 MNDR
31 CTNM
32 MAMO
33 ZBAO
34 BOLD
35 MMA
36 UBXG
37 IBAC
38 AUNA
39 BKHA
40 LOBO
41 RDDT
42 ALAB
43 INTJ
44 RYDE
45 LGCL
46 SMXT
47 VHAI
48 DYCQ
49 CHRO
50 UMAC
51 HLXB
52 MGX
53 TBBB
54 TELO
55 KYTX
56 PMNT
57 AHR
58 LEGT
59 ANRO
60 GUTS
61 AS
62 FBLG
63 AVBP
64 BTSG
65 HAO
66 CGON
67 YIBO
68 JL
69 SUGP
70 JVSA
71 KSPI
72 CCTG
73 PSBD
74 SYNX
75 SDHC
76 ROMA


In [123]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23953 entries, 0 to 23952
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Open                           23953 non-null  float64
 1   High                           23953 non-null  float64
 2   Low                            23953 non-null  float64
 3   Close                          23953 non-null  float64
 4   Volume                         23953 non-null  int64  
 5   Dividends                      23953 non-null  float64
 6   Stock Splits                   23953 non-null  float64
 7   Ticker                         23953 non-null  object 
 8   Year                           23953 non-null  int64  
 9   Month                          23953 non-null  int64  
 10  Weekday                        23953 non-null  int64  
 11  Date                           23953 non-null  object 
 12  growth_1d                      23876 non-null 

In [124]:
stocks_df.transpose()

,0,1,2,3,4,5,6,7,8,9,...,23943,23944,23945,23946,23947,23948,23949,23950,23951,23952
Open,4.0,2.99,2.53,2.91,2.94,2.67,2.74,2.65,2.9,2.97,...,3.75,2.85,2.75,3.021,2.87,2.84,2.85,2.883,2.96,2.78
High,4.2,3.11,3.11,3.09,2.94,2.9,2.74,3.1,3.0,3.0,...,3.955,3.31,3.153,3.05,2.89,3.0,2.935,2.9,3.12,2.9268
Low,2.8,2.35,2.41,2.6,2.41,2.65,2.56,2.55,2.55,2.65,...,2.6,2.77,2.75,2.85,2.56,2.64,2.79,2.73,2.7,2.7
Close,3.02,2.66,2.92,2.73,2.69,2.72,2.7,2.8,2.92,2.78,...,2.86,2.77,3.0,2.88,2.66,2.875,2.795,2.79,2.78,2.8
Volume,440600,147300,73800,51100,56500,27500,27000,95300,91400,69600,...,381700,52500,37400,36300,123100,63100,10200,33600,123300,158043
Dividends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stock Splits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ticker,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,...,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA
Year,2024,2024,2024,2024,2024,2024,2024,2024,2024,2024,...,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025
Month,5,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [125]:
stocks_df['Sharpe'] = (stocks_df['growth_252d'] - 0.045) / stocks_df['volatility']

In [126]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [127]:
stocks_df_filtered = stocks_df[stocks_df['Date'] == pd.Timestamp('2025-06-06')].reset_index(drop=True)

In [128]:
stocks_df_filtered.transpose()

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
Open,16.26,36.439999,20.43,10.51,0.38,1.11,1.21,26.65,10.56,7.71,...,1.52,5.31,0.75,10.85,85.68,1.075,13.9,1.75,19.9,3.65
High,17.200001,36.540001,20.440001,10.51,0.386,1.27,1.32,26.65,10.56,7.78,...,1.52,5.355,0.805,10.85,86.32,1.1,14.0,1.75,20.15,3.95
Low,15.55,35.93,19.969999,10.51,0.201,1.1,1.161,25.924999,10.56,7.25,...,1.37,5.289,0.75,10.85,84.425003,1.075,13.75,1.68,19.27,3.63
Close,15.88,36.389999,20.41,10.51,0.295,1.255,1.29,26.389999,10.56,7.32,...,1.37,5.32,0.8,10.85,85.93,1.095,13.88,1.68,19.27,3.7
Volume,431200,137900,1300,0,3428900,501000,21500,580300,1800,123700,...,46500,7700,9100,228100,223200,1600,56200,10000,38100,84400
Dividends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stock Splits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ticker,NAKA,BOW,HDL,RFAI,JDZG,RAY,BTOC,ZK,GPAT,PAL,...,YIBO,JL,SUGP,JVSA,KSPI,CCTG,PSBD,SYNX,SDHC,ROMA
Year,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025,...,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025
Month,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [129]:
stocks_df_filtered[['Close','Ticker','growth_future_30d','Date']].transpose()

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
Close,15.88,36.389999,20.41,10.51,0.295,1.255,1.29,26.389999,10.56,7.32,...,1.37,5.32,0.8,10.85,85.93,1.095,13.88,1.68,19.27,3.7
Ticker,NAKA,BOW,HDL,RFAI,JDZG,RAY,BTOC,ZK,GPAT,PAL,...,YIBO,JL,SUGP,JVSA,KSPI,CCTG,PSBD,SYNX,SDHC,ROMA
growth_future_30d,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Date,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,...,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00,2025-06-06 00:00:00


In [130]:
stocks_df_filtered[['growth_252d','Sharpe']].describe()

,growth_252d,Sharpe
count,73.000000,73.000000
mean,1.227946,0.284576
std,1.480238,0.512601
min,0.024970,-0.079677
25%,0.293510,0.040265
50%,0.763188,0.083768
75%,1.446667,0.291048
max,8.097413,2.835668


## Q3 ‘Fixed Months Holding Strategy’

In [131]:
# Calculate future growth for each stock over the next 1 to 12 months

for ticker in stocks_df['Ticker'].unique():
    mask = stocks_df['Ticker'] == ticker
    for i in range(1, 13):  # 1 to 12 months
        col_name = f'growth_future_{i}m'
        # Calculate growth over future 21*i trading days
        stocks_df.loc[mask, col_name] = (
            stocks_df.loc[mask, 'Close'].shift(-21 * i) / stocks_df.loc[mask, 'Close']
        )

In [132]:
stocks_df.transpose()


,0,1,2,3,4,5,6,7,8,9,...,23943,23944,23945,23946,23947,23948,23949,23950,23951,23952
Open,4.0,2.99,2.53,2.91,2.94,2.67,2.74,2.65,2.9,2.97,...,3.75,2.85,2.75,3.021,2.87,2.84,2.85,2.883,2.96,2.78
High,4.2,3.11,3.11,3.09,2.94,2.9,2.74,3.1,3.0,3.0,...,3.955,3.31,3.153,3.05,2.89,3.0,2.935,2.9,3.12,2.9268
Low,2.8,2.35,2.41,2.6,2.41,2.65,2.56,2.55,2.55,2.65,...,2.6,2.77,2.75,2.85,2.56,2.64,2.79,2.73,2.7,2.7
Close,3.02,2.66,2.92,2.73,2.69,2.72,2.7,2.8,2.92,2.78,...,2.86,2.77,3.0,2.88,2.66,2.875,2.795,2.79,2.78,2.8
Volume,440600,147300,73800,51100,56500,27500,27000,95300,91400,69600,...,381700,52500,37400,36300,123100,63100,10200,33600,123300,158043
Dividends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stock Splits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ticker,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,...,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA
Year,2024,2024,2024,2024,2024,2024,2024,2024,2024,2024,...,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025
Month,5,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [133]:
min_dates = stocks_df.groupby('Ticker')['Date'].min().reset_index()
min_dates

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
...,...,...
72,YIBO,2024-01-25
73,YYGH,2024-04-22
74,ZBAO,2024-04-02
75,ZK,2024-05-10


In [134]:
# min_date_df = pd.merge(min_dates, stocks_df[['Ticker', 'Date','growth_future_1m',
#        'growth_future_2m', 'growth_future_3m', 'growth_future_4m',
#        'growth_future_5m', 'growth_future_6m', 'growth_future_7m',
#        'growth_future_8m', 'growth_future_9m', 'growth_future_10m',
#        'growth_future_11m', 'growth_future_12m']], on=['Ticker', 'Date'], how='inner')

In [135]:
stocks_ipomerged_df = stocks_df.merge(min_dates, on='Ticker', how='left')


In [136]:
#stocks_df = stocks_df.rename(columns={ 'Date_y': 'Min_Date'})

In [137]:
stocks_ipomerged_df.transpose()

,0,1,2,3,4,5,6,7,8,9,...,23943,23944,23945,23946,23947,23948,23949,23950,23951,23952
Open,4.0,2.99,2.53,2.91,2.94,2.67,2.74,2.65,2.9,2.97,...,3.75,2.85,2.75,3.021,2.87,2.84,2.85,2.883,2.96,2.78
High,4.2,3.11,3.11,3.09,2.94,2.9,2.74,3.1,3.0,3.0,...,3.955,3.31,3.153,3.05,2.89,3.0,2.935,2.9,3.12,2.9268
Low,2.8,2.35,2.41,2.6,2.41,2.65,2.56,2.55,2.55,2.65,...,2.6,2.77,2.75,2.85,2.56,2.64,2.79,2.73,2.7,2.7
Close,3.02,2.66,2.92,2.73,2.69,2.72,2.7,2.8,2.92,2.78,...,2.86,2.77,3.0,2.88,2.66,2.875,2.795,2.79,2.78,2.8
Volume,440600,147300,73800,51100,56500,27500,27000,95300,91400,69600,...,381700,52500,37400,36300,123100,63100,10200,33600,123300,158043
Dividends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stock Splits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ticker,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,NAKA,...,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA,ROMA
Year,2024,2024,2024,2024,2024,2024,2024,2024,2024,2024,...,2025,2025,2025,2025,2025,2025,2025,2025,2025,2025
Month,5,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [141]:
stocks_ipo_return_df = stocks_ipomerged_df[stocks_ipomerged_df['Date_x'] == stocks_ipomerged_df['Date_y']].reset_index()

In [161]:
stocks_ipo_return_df.transpose()

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
index,0,265,535,809,1050,1326,1602,1879,2158,2402,...,20413,20766,21120,21474,21796,22153,22510,22868,23229,23589
Open,4.0,23.0,27.0,10.0,4.8,5.05,5.0,26.0,9.98,15.0,...,4.0,6.849315,4.38,10.05,86.883162,5.75,14.039482,3.99,23.82,10.0
High,4.2,24.27,30.0,10.08,5.4,6.04,6.26,29.360001,10.0,15.24,...,4.0,9.881278,4.72,10.05,90.142456,9.833,14.220914,4.0,24.67,11.8
Low,2.8,22.139999,19.799999,10.0,3.595,4.02,4.139,25.51,9.98,14.74,...,2.4,5.607306,3.4,10.03,85.230032,5.03,13.83213,3.26,23.5,2.9
Close,3.02,23.799999,22.290001,10.04,4.07,4.23,4.62,28.26,9.99,14.75,...,2.79,6.79452,4.0,10.03,90.142456,7.7,13.987644,3.61,24.15,2.9
Volume,440600,3335800,670000,1175800,2366200,2379300,1323900,9709800,786200,3615600,...,875500,1361900,1782000,351800,3410400,5181000,1044800,711400,426100,4601100
Dividends,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stock Splits,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Ticker,NAKA,BOW,HDL,RFAI,JDZG,RAY,BTOC,ZK,GPAT,PAL,...,YIBO,JL,SUGP,JVSA,KSPI,CCTG,PSBD,SYNX,SDHC,ROMA
Year,2024,2024,2024,2024,2024,2024,2024,2024,2024,2024,...,2024,2024,2024,2024,2024,2024,2024,2024,2024,2024


In [162]:
stock_ipo_filtered_df = stocks_ipo_return_df[['growth_future_1m',
       'growth_future_2m', 'growth_future_3m', 'growth_future_4m',
       'growth_future_5m', 'growth_future_6m', 'growth_future_7m',
       'growth_future_8m', 'growth_future_9m', 'growth_future_10m',
       'growth_future_11m', 'growth_future_12m']]

In [165]:
stock_ipo_filtered_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
growth_future_1m,77.0,0.926390,0.342767,0.098947,0.775236,0.977000,1.047796,2.646505
growth_future_2m,77.0,0.936866,0.568481,0.073800,0.680193,1.000000,1.139076,4.874759
growth_future_3m,77.0,0.834120,0.410362,0.060947,0.508139,0.927500,1.075676,2.040000
growth_future_4m,77.0,0.822711,0.401473,0.045368,0.515385,0.909091,1.125316,1.605000
growth_future_5m,77.0,0.802881,0.486716,0.054109,0.445378,0.821092,1.017822,3.213873
growth_future_6m,77.0,0.864347,0.649114,0.061432,0.387417,0.802239,1.103896,3.670520
growth_future_7m,77.0,0.846270,0.706456,0.048274,0.324100,0.844875,1.150909,5.122350
growth_future_8m,77.0,0.829971,0.754255,0.043103,0.225974,0.812109,1.074739,5.171484
growth_future_9m,77.0,0.878868,0.925133,0.033144,0.232468,0.822715,1.037624,6.764933
growth_future_10m,76.0,0.913269,0.900344,0.041357,0.248485,0.772592,1.194894,5.352601
